<a href="https://www.kaggle.com/code/aleksandrmorozov123/bitcoin-python-programming?scriptVersionId=99431546" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Exercises to understand bitcoin from the basics, to better understand the inner workings of bitcoin. Then I plan to do an analytical project.**

**First step - finite fields (mathematics)**

In [2]:
class FieldElement:
    def __init__ (self, num, prime):
        if num >= prime or num < 0 :
            error = 'Num {} not in the field range 0 to {}'.format (num, prime - 1)
            raise ValueError (error)
        self.num = num
        self.prime = prime

    def __repr__ (self):
        return 'FieldElement_{}({})'.format (self.prime, self.num)
    
    def __eq__ (self, other):
        if other is None:
            return False
        return self.num == other.num and self.prime == other.prime
    
    def __add__ (self, other):
        if self.prime != other.prime:
            raise TypeError ('Cannot add two numbers in different Fields')
        num = (self.num + other.num) % self.prime
        return self.__class__ (num, self.prime)
    
    def __sub__ (self, other):
        if self.prime != other.prime:
            raise TypeError ('Cannot subtract two numbers in different Fields')
        num = (self.num - other.num) % self.prime
        return self.__class__ (num, self.prime)
    
    def __pow__ (self, exponent):
        num = (self.num ** exponent) % self.prime
        return self.__class__ (num, self.prime)
        n = exponent
        while n < 0:
            n += self.prime - 1
        num = pow (self.num, n, self.prime)
        return self.__class__ (num, self.prime)
        n = exponent % (self.prime - 1)
        num = pow (self.num, n, self.prime)
        return self.__class__ (num, self.prime)
    
    def __ne__ (self, other):
        return not (self == other)
    
    def __mul__ (self, other):
        if self.prime != other.prime:
            raise TypeError ("Cannot multiply two numbers in different Fields")
        num = (self.num * other.num) % self.prime
        return self.__class__ (num, self.prime)
    
    def __truediv__ (self, other):
        if self.prime != other.prime:
            raise TypeError ('Cannot divide two numbers in different Fields')
        num = self.num * pow (other.num, self.prime - 2, self.prime) % self.prime
        return self.__class__ (num, self.prime)

In [3]:
a = FieldElement (5, 19)
b = FieldElement (8, 25)
print (a == b)

False


In [4]:
print (a == a)

True


In [5]:
a = FieldElement (5, 19)
b = FieldElement (13, 19)
c = FieldElement (6, 19)
print(a + b == c)

False


In [6]:
a = FieldElement (12, 15)
b = FieldElement (5, 16)
print (a ** 3 == b)

False


In [7]:
a = FieldElement (8, 19)
b = FieldElement (4, 12)
print (a ** - 5 == b)

False


**Elliptic curves**

In [8]:
class Point:
    def __init__ (self, x, y, a, b):
        self.a = a
        self.b = b
        self.x = x
        self.y = y
        if self.x is None and self.y is None:
            return
        if self.y ** 2 != self.x ** 3 + a * x + b:
            raise ValueError ('({}, {}) is not on the curve'.format (x, y))
        
    def __eq__ (self, other):
        if self.x == other.x and self.y != other.y:
            return self.__class__ (None, None, self.a, self.b)
        return self.x == other.x and self.y == other.y \
            and self.a == other.a and self.b == other.b
        
        if self.y ** 2 != self.x ** 3 + a * x + b:
            raise ValueError ('({}, {}) is not on the curve'.format (x, y))
        
    def __add__ (self, other):
        if self.a != other.a or self.b != other.b:
            raise TypeError ('Points {}, {} are not on same curve'.format (self, other))
        if self.x is None:
            return other
        if other.x is None:
            return self
        if self.x != other.x:
            s = (other.y - self.y) / (other.x - self.x)
            x = s**2 - self.x - other.x
            y = s * (self.x - x) - self.y
            return self.__class__ (x, y, self.a, self.b)
        if self == other:
            s = (3 * self.x**2 + self.a) / (2 * self.y)
            x = s**2 - 2 * self.x
            y = s * (self.x - x) - self.y
            return self.__class__ (x, y, self.a, self.b)
        
    def __rmul__ (self, coefficient):
        product = self.__class__ (None, None, self.a, self.b)
        for _ in range (coefficient):
            product += self
        return product
        coef = coefficient
        current = self
        result = self.__class__ (None, None, self.a, self.b)
        while coef:
            if coef & 1:
                result += current
            current += current
            coef >>= 1
        return result
    
    def __ne__ (self, other):
        return not (self == other)
        

In [9]:
p1 = Point (-1, -1, 3, 5)
p2 = Point (-1, 1, 3, 5)
inf = Point (None, None, 3, 5)
print (p1 + inf)

In [10]:
print (inf + p2)

In [11]:
print (p1 + p2)

**Elliptic curve over Finite Fields**

In [12]:
a = FieldElement (num = 0, prime = 223)
b = FieldElement (num = 7, prime = 223)
x = FieldElement (num = 192, prime = 223)
y = FieldElement (num = 105, prime = 223)
p1 = Point (x, y, a, b)
print (p1)

**Point addition over Finite Fields**

**Scalar multiplicationn Redux**

In [13]:
prime = 223
a = FieldElement (num = 0, prime = prime)
b = FieldElement (num = 7, prime = prime)
x1 = FieldElement (num = 192, prime = prime)
y1 = FieldElement (num = 105, prime = prime)
x2 = FieldElement (num = 17, prime = prime)
y2 = FieldElement (num = 56, prime = prime)
p1 = Point (x1, y1, a, b)
p2 = Point (x2, y2, a, b)
print (p1 + p2)

In [14]:
prime = 223
a = FieldElement (0, prime)
b = FieldElement (7, prime)
def on_curve (x, y):
    return y**2 == x**3 + a*x + b
print (on_curve (FieldElement (192, prime), FieldElement (105, prime)))

print (on_curve (FieldElement (17, prime), FieldElement (56, prime)))

print (on_curve (FieldElement (200, prime), FieldElement (119, prime)))

print (on_curve (FieldElement (1, prime), FieldElement (193, prime)))

print (on_curve (FieldElement (42, prime), FieldElement (99, prime)))

True
True
False
True
False


**Coding scalar multiplication**

In [15]:
prime = 223
a = FieldElement (0, prime)
b = FieldElement (7, prime)
def on_curve (x, y):
    return y**2 == x**3 + a*x + b
print (on_curve (FieldElement (192, prime), FieldElement (105, prime)))

print (on_curve (FieldElement (17, prime), FieldElement (56, prime)))

print (on_curve (FieldElement (200, prime), FieldElement (119, prime)))

print (on_curve (FieldElement (42, prime), FieldElement (99, prime)))

True
True
False
False


**Working with secp256k1**

In [16]:
gx = 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798
gy = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
p = 2**256 - 2**32 - 977
print (gy**2 % p == (gx**3 +7) % p)

True


In [17]:
P = 2**256 - 2**32 - 977

class S256Field (FieldElement):
    def __init__ (self, num, prime = None):
        super ().__init__ (num = num, prime = P)
        
    def __repr__ (self):
        return '{:x}'.format (self.num).zfill (64)
    
    def sqrt (self):
        return self ** ((P + 1) // 4)

In [18]:
A = 0
B = 7
N = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141

class S256Point (Point):
    def __init__ (self, x, y, a = None, b = None):
        a, b = S256Field (A), S256Field (B)
        if type (x) == int:
            super ().__init__(x = S256Field (x), y = S256Field (y), a = a, b = b)
        else:
            super ().__init__ (x = x, y = y, a = a, b = b)
            
    def __rmul__ (self, coefficient):
        coef = coefficient % N
        return super ().__rmul__(coef)
    
    def verify (self, z, sig):
        s_inv = pow (sig.s, N - 2, N)
        u = z * s_inv % N
        v = sig.r * s_inv & N
        total = u * G + v * self
        return total.x.num == sig.r
    
    def verify (self, z, sig):
        s_inv = pow (sig.s, N - 2, N)
        u = z * s_inv % N
        v = sig.r * s_inv % N
        total = u * G + v * self
        return total.x.num == sig.r
    
    def sec (self):
        """return the binary version of the SEC format"""
        return b'\x04' + self.x.num.to_bytes (32, 'big') + self.y.num.to_bytes (32, 'big')
    
    def sec (self, compressed = True):
        """returns the binary versiom of the SEC format"""
        if compressed:
            if self.y.num % 2 == 0:
                return b'\x02' + self.x.num.to_bytes (32, 'big')
            else:
                return b'\x03' + self.x.num.to_bytes (32, 'big')
        else:
            return b'\x04' + self.x.num.to_bytes (32, 'big') + self.y.num.to_bytes (32, 'big')
        
        @ classmethod
        def parse (self, sec_bin):
            """returns a Point object from a SEC binary (not hex)"""
            if sec_bin [0] == 4:
                x = int.from_bytes (sec_bin [1:33], 'big')
                y = int.from_bytes (sec_bin [33:65], 'big')
                return S256Point (x = x, y = y)
            is_even = sec_bin [0] == 2
            x = S256Field (int.from_bytes (sec_bin [1:], 'big'))
            # right side of the equation y^2 = x^3 + 7
            alpha = x ** 3 + S256Field (B)
            # solve for left side
            beta = aplpha.sqrt ()
            if beta.num % 2 == 0:
                even_beta = beta
                odd_beta = S256Field (P - beta.num)
            else:
                even_beta = S256Field (P - beta.num)
                odd_beta = beta
            if is_even:
                return S256Point (x, even_beta)
            else:
                return S256Point (x, odd_beta)
            
    def hash160 (self, compressed = True):
        return hash160 (self.sec (compressed))
    
    def adress (self, comressed = True, testnet = False):
        """Returns the address string"""
        h160 = self.hash160 (compressed)
        if testnet:
            prefix = b'\x6f'
        else:
            prefix = b'\x00'
        return encode_base58_checksum (prefix + h160)

In [19]:
G = S256Point (0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798,
               0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8)
print (N * G)

**Verifying a signature**

In [20]:
class Signature:
    def __init__ (self, r, s):
        self.r = r
        self.s = s
        
    def __repr__ (self):
        return "Signature ({:x}, {:x})".format (self.r, self.s)
    
    def der (self):
        rbin = self.r.to_bytes (32, byteorder = 'big')
        # remove all null bytes at the beginng
        rbin = rbin.lstrip (b'\x00')
        # if rbin has a high bit, add a \x00
        if rbin [0] & 0x80:
            rbin = b'\x00' + rbin
        result = bytes ([2, len (rbin)]) + rbin
        sbin = self.s.to_bytes (32, byteorder = 'big')
        # remove all null bytes at the beginning
        sbin = sbin.lstrip (b'\x00')
        # if sbin has a high bit, add a \x00
        if sbin [0] & 0x80:
            sbin = b'\x00' + sbin
        result += bytes ([2, len (sbin)]) + sbin
        return bytes ([0x30, len (result)]) + result

In [21]:
class PrivateKey:
    def __init__ (self, secret):
        self.secret = secret
        self.point = secret * G
        
    def hex (self):
        return '{:x}'.format (self.secret).zfill (64)
    
    def sign (self, z):
        k = randint (0, N)
        r = (k * G).x.num
        k_inv = pow (k, N - 2, N)
        s = (z + r * self.secret) * k_inv % N
        if s > N/2:
            s = N - s
        return Signature (r, S)
        k = self.deterministic_k (z)
        r = (k * G).x.num
        k_inv = pow (k, N - 2, N)
        s = (z + r * self.secret) * k_inv % N
        if s > N / 2:
            s = N - s
        return Signature (r, s)
    
    def deterministic_k (self, z):
        k = b'\x00' * 32
        v = b'\x01' * 32
        if z > N:
            z -= N
        z_bytes = z.to_bytes (32, 'big')
        secret_bytes = self.secret.to_bytes (32, 'big')
        s256 = hashlib.sha256
        k = hmac.new (k, v + b'\x00' + secret_bytes + z_bytes, s256).digest ()
        v = hmac.new (k, v, s256).digest ()
        k = hmac.new (k, v + b'\x01' + secret_bytes + z_bytes, s256).digest ()
        v = hmac.new (k, v, s256).digest ()
        while True:
            v = hmac.new (k, v, s256).digest ()
            candidate = int.from_bytes (v, 'big')
            if candidate >= 1 and candidate < N:
                return candidate
            k = hmac.new (k, v + b'\x00', s256).digest ()
            v = hmac.new (k, v, s256).digest ()
            
    def wif (self, compressed = True, testnet = False):
        secret_bytes = self.secret.to_bytes (32, 'big')
        if testnet:
            prefix = b'\xef'
        else:
            prefix = b'\x80'
        if compressed:
            suffix = b'\x01'
        else:
            suffix = b''
        return encode_base58_checksum (prefix + secret_bytes + suffix)

In [22]:
BASE58_ALPHABET = '123456789ABCDEFGHJKLMNPQRSTUVWXYZabcdefghijkmnopqrstuvwxyz'

def encode_base58 (s):
    count = 0
    for c in s:
        if c == 0:
            count += 1
        else:
            break
    num = int.from_bytes (s, 'big')
    prefix = '1' * count
    result = ''
    while num > 0:
        num, mod = divmod (num, 58)
        result = BASE58_ALPHABET [mod] + result
    return prefix + result

In [23]:
def encode_base58_checksum (b):
    return encode_base58 (b + hash256(b) [:4])

**Serialization**

In [24]:
r = 0x37206a0610995c58074999cb9767b87af4c4978db68c06e8e6e81d282047a7c6
s = 0x8ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec
sig = Signature (r, s)
print (sig.der ().hex ())

3045022037206a0610995c58074999cb9767b87af4c4978db68c06e8e6e81d282047a7c60221008ca63759c1157ebeaec0d03cecca119fc9a75bf8e6d0fa65c841c8e2738cdaec


In [25]:
h = '7c076ff316692a3d7eb3c3bb0f8b1488cf72e1afcd929e29307032997a838a3d'
print (encode_base58 (bytes.fromhex (h)))

9MA8fRQrT4u8Zj8ZRd6MAiiyaxb2Y1CMpvVkHQu5hVM6


In [26]:
h = 'eff69ef2b1bd93a66ed5219add4fb51e11a840f404876325a1e8ffe0529a2c'
print (encode_base58 (bytes.fromhex (h)))

4fE3H2E6XMp4SsxtwinF7w9a34ooUrwWe4WsW1458Pd


In [27]:
h = 'c7207fee197d27c618aea621406f6bf5ef6fca38681d82b2f06fddbdce6feab6'
print (encode_base58 (bytes.fromhex (h)))

EQJsjkd6JaGwxrjEhfeqPenqHwrBmPQZjJGNSCHBkcF7


**Coding transactions**

In [28]:
class Tx:
    def __init__ (self, version, tx_ins, tx_outs, locktime, testnet = False):
        self.version = version
        self.tx_ins = tx_ins
        self.tx_outs = tx_outs
        self.locktime = locktime
        self.testnet = testnet
        
    def __repr__ (self):
        tx_ins = ''
        for tx_in in self.tx_ins:
            tx_ins += tc_in.__repr__ () + '\n'
        tx_outs = ''
        for tx_out in self.tx_outs:
            tx_outs += tx.out.__repr__ () + '\n'
        return 'tx: {}\nversion: {}\ntx_ins:\n{}tx_outs:\n{}locktime: {}'.format (self.id (),
                                                                                 self.version, tx_ins,
                                                                                 tx_outs, self.locktime)
    
    def id (self):
        """Human-readable hexasecimal of the transactions hash"""
        return self.hash ().hex ()
    
    def hash (self):
        """Binary hash of the legacy sereialization"""
        return hash256 (self.serialize ())[::-1]
    
    @classmethod
    def parse (cls, stream):
        serialized_version = stream.read (4)
        
    def parse (cls, s, testnet = False):
        version = little_endian_to_int (s.read (4))
        num_inputs = read_varint (s)
        inputs = []
        for _ in range (num_inputs):
            inputs.append (TxIn.parse (s))
        num_outputs= read_varint (s)
        outputs = []
        for _ in range (num_outputs):
            outputs.append (TxOut.parse (s))
        locktime = little_to_endian_to_int (s.read (4))
        return cls (version, inputs, outputs, locktime, testnet = testnet)
        
    def serialize (self):
        """Returns the byte serialization of the transaction"""
        result = int_to_little_endian (self.version, 4)
        result += encode_variant (len (self.tx_ins))
        for tx_in in self.tx_ins:
            result += tx_in.serialie ()
        result += encode_variant (len(self.tx_outs))
        for tx_out in self.tx_outs:
            result += tx_out.serialize ()
        result += int_to_little_endian (self.locktime, 4)
        return result
    
    def fee (self, testnet = False):
        input_sum, output_sum = 0, 0
        for tx_in in self.tx_ins:
            input_sum += tx_in.value (testnet = testnet)
        for tx_out in self.tx_outs:
            output_sum += tx_out.amount
        return imput_sum - output_sum
    
    def fee (self):
        """Returns the fee of this transaction in satoshi"""
        input_sum, output_sum = 0, 0
        for tx_in in self.tx_ins:
            input_sum += tx_in.value (self.testnet)
        for tx_out in self.tx_outs:
            output_sum += tx_out.amount
        return input_sum - output_sum

In [29]:
class TxIn:
    def __init__ (self, prev_tx, prev_index, script_sig = None, sequence = 0xffffffff):
        self.prev_tx = prev_tx
        self.prev_index = prev_index
        if script_sig is None:
            self.script_sig = Sript ()
        else:
            self.svript_sig = script_sig
        self.sequence = sequence
        
    def __repr__ (self):
        return '{}:{}'.format (self.prev_tx.hex (), self.prev_index)
    
    def serialize (self):
        """Returns the byte serialization of the transaction input"""
        result = self.prev_tx [::-1]
        result += int_to_little_endian (self.prev_index, 4)
        result += self.script_sig_serialize ()
        result += int_to_little_endian (self.sequence, 4)
        return result
    
    def fetch_tx (self, testnet = False):
        return TxFetcher.fetch (self.prev_tx.hex (), testnet = testnet)
    
    def value (self, testnet = False):
        """Get the output value by looking uo the tx hash. Returns the amount in satoshi"""
        tx = self.fetch_tx (testnet = testnet)
        return tx.tx_outs [self.prev_index].amount
    
    def script_pubkey (self, testnet = False):
        """Get the ScriptPubKey by looking up the tx hash. Returns a Script object"""
        tx = self.fetch_tx (testnet = testnet)
        return tx.tx_outs [self.prev_index].script_pubkey
    
    @classmethod
    def parse (cls, s):
        """take a byte stream and parses the rx_input at the start. Returns a TxIn object"""
        prev_tx = s.read (32)[::-1]
        prev_index = litte_endian_to_int (s.read (4))
        script_sig =Script.parse (s)
        sequence = little_endian_to_int (s.read (4))
        return cls (prev_tx, prev_index, script_sig, sequence)

In [30]:
class TxOut:
    def __init__ (self, amount, script_pubkey):
        self.amount = amount
        self.script_pubkey = script_pubkey
        
    def __repr__ (self):
        return '{}:{}'.format (self.amount, self.script_pubkey)
    
    def serialize (self):
        """Returns the byte serialization of the transaction output"""
        result = int_to_little_endian (self.amount, 8)
        result += self.script_pubkey.serialize ()
        return result
    
    @classmethod
    def parse (cls, s):
        """Takes a byte stream and parses the tx_output at the start. Returns a TxOut object."""
        amount = little_endian_to_int (s.read (8))
        script_pubkey = Script.parse (s)
        return cls (amount, script_pubkey)

In [31]:
class TxFetcher:
    cache = {}
    
    @classmethod
    def get_url (cls, testnet = False):
        if testnet:
            return 'http://testnet.programmingbitcoin.com'
        else:
            return 'http://mainnet.programmingbitcoin.com'
        
    @classmethod
    def fetch (cls, tx_id, testnet = False, fresh = False):
        if fresh or (tx_id not in cls.cache):
            url = '{}/tx/{}.hex'.format (cls.get_url (testnet), tx_id)
            response = requests.get (url)
            try:
                raw = bytes.fromhex (response.text.strip ())
            except ValueErrror:
                raise ValueError ('unexpected response: {}'.format (response.text))
            if raw [4] == 0:
                raw = raw [:4] + raw [6:]
                tx = Tx.parse (BytesIO (raw), testnet = testnet)
                tx.locktime = little_endian_to_int (rae[-4:])
            else:
                tx =Tx.parse (BytesIO (raw), testnet = testnet)
            if tx.id () != tx_id:
                raise ValueError ('not the same if: {} vs {}'.format (tx.id (), tx_id))
            cls.cache [tx_id] = tx
        cls.cache [tx_id].testnet = testnet
        return cls.cache [tx_id]

In [32]:
def little_endian_to_int(b):
    '''little_endian_to_int takes byte sequence as a little-endian number. Returns an integer'''
    return int.from_bytes(b, 'little')
def int_to_little_endian(n, length):
    '''endian_to_little_endian takes an integer and returns the little-endian byte sequence of length'''
    return n.to_bytes(length, 'little')

In [33]:
def read_varint (s):
    """read varint reads a variable integer from a stream"""
    i = s.read (1)[0]
    if i == 0xfd:
        # 0xfd means the next two bytes are the number
        return little_endian_to_int (s.read (2))
    elif i == 0xfe:
        # 0xfe means the next four bytes are the number
        return little_endian_to_int (s.read (4))
    elif i == 0xff:
        # 0xff means the next eight bytes are the number 
        return little_endian_to_int (s.read (8))
    else:
        # anything else is just the integer
        return i
    
def encode_varint (i):
    """encodes an integer as a varint"""
    if i < 0xfd:
        return bytes ([i])
    elif i < 0x10000:
        return b'\xfd' + int_to_little_endian (i, 2)
    elif i < 0x100000000:
        return b'\xfe' + int_to_little_endian (i, 4)
    elif i < 0x10000000000000000:
        return b'\xff' + int_to_little_endian (i, 8)
    else:
        raise ValueError ('integer too large: {}'.format (i))

**Coding Opcodes**

In [34]:
def op_dup (stack):
    if len (stack) < 1:
        return False
    stack.append (stack [-1])
    return True

OP_CODE_FUNCTIONS = {118: op_dup}

def op_hash256 (stack):
    if len (stack) < 1:
        return False
    element = stack.pop ()
    stack.append(hash256 (element))
    return True

OP_CODE_FUNCTIONS = {170: op_hash256}

def op_hash160 (stack):
    if len (stack) < 1:
        return False
    element = stack.pop ()
    h160 = hash160 (element)
    stack.append (h160)
    return True

def op_checksig (stack, z):
    if len (stack) < 2:
        return False
    sec_pubkey = stack.pop ()
    der_signature = stack.pop () [:-1]
    try:
        point = S256Point.parse (sec_pubkey)
        sig = Signature.parse (der_signature)
    except (ValueError, SyntaxError) as e:
        return False
    if point.verify (z, sig):
        stack.append (encode_num (1))
    else:
        stack.append (encode_num (0))
    return True

**Coding a Script Parser and Serializer**

In [35]:
class Script:
    def __init__ (self, cmds = None):
        if cmds is None:
            self.cmds = []
        else:
            self.cmds = cmds
    
    @classmethod
    def parse (cls, s):
        length = read_varint (s)
        cmds = []
        count = 0
        while count < lenght:
            current = s.read (1)
            count += 1
            current_byte = current [0]
            if current_byte >= 1 and current_byte <= 75:
                n = current_byte
                cmds.append (s.read (n))
                count += n
            elif current_byte == 76:
                data_length = little_endian_to_int (s.read (1))
                cmds.append (s.read (data_length))
                count += data_length + 1
            elif current_byte == 77:
                data_length = little_endian_to_int (s.read (2))
                cmds.append (s.read (data_length))
                count += data_length + 2
            else:
                op_code = current_byte
                cmds.append (op_code)
        if count != length:
            raise SyntaxError ('parsing script failed')
        return cls (cmds)
    
    def raw_serialize (self):
        result = b''
        for cmd in self.cmds:
            if type (cmd) == int:
                result += int_to_little_endian (cmd, 1)
            else:
                length = len (cmd)
                if length < 75:
                    result += int_to_little_endian (length, 1)
                elif length > 75 and length < 0x100:
                    result += int_to_little_endian (76, 1)
                    result += int_to_little_endian (length, 1)
                elif length >= 0x100 and length <= 520:
                    result += int_to_little_endian (77, 1)
                    result += int_to_little_endian (length, 2)
                else:
                    raise ValueError ('too long an cmd')
                result += cmd
            return result
        
    def serialize (self):
        result = self.raw_serialize ()
        total = len (result)
        return encode_varint (total) + result
    
    def __add__ (self, other):
        return Script (self.cmds + other.cmds)
    
    def evaluate (self, z):
        cmds = self.cmds [:]
        stack = []
        altstack = []
        while len (cmds) > 0:
            cmd = cmds.pop (0)
            if type (cmd) == int:
                operation = OP_CODE_FUNCTIONS [cmd]
                if cmd in (99, 100):
                    if not operation (stack, cmds):
                        LOGGER.info ('bad op: {}'.format (OP_CODE_NAMES [cmd]))
                        return False
                elif cmd in (107, 108):
                    if not operation (stack, altstack):
                        LOGGER.info ('bad op: {}'.format (OP_CODE_NAMES [cmd]))
                        return False
                elif cmd in (172, 173, 174, 175):
                    if not operation (stack, z):
                        LOGGER.info ('bad op: {}'.format (OP_CODE_NAMES [cmd]))
                        return False
                else:
                    if not operation (stack):
                        LOGGER.info ('bad op: {}'.format (OP_CODE_NAMES [cmd]))
                        return False
            else:
                stack.append (cmd)
        if len (stack) == 0:
            return False
        if stack.pop () == b'':
            return False
        return True

In [36]:
def encode_num (num):
    if num == 0:
        return b''
    abs_num = abs (num)
    negative = num < 0
    result = bytearray ()
    while abs_num:
        result.append (abs_num & 0xff)
        abs_num >>= 8
    if result [-1] & 0x80:
        if negative:
            result.append (0x80)
        else:
            result.append (0)
    elif negative:
        result [-1] |= 0x80
    return bytes (result)

def decode_num (element):
    if element == b'':
        return 0
    big_endian = element [::-1]
    if big_endian [0] & 0x80:
        negative = True
        result = big_endian [0] & 0x7f
    else:
        negative = False
        result = big_endian [0]
    for c in big_endian [1:]:
        result <<= 8
        result += c
    if negative:
        return - result
    else:
        return result
    
def op_0 (stack):
    stack.append (encode_num (0))
    return True